In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
titanic = sns.load_dataset('titanic')
iris = sns.load_dataset('iris')
mpg = sns.load_dataset('mpg')



# 1. Iris


a. 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width),
꽃받침길이(petal_length), 꽃받침폭(petal_width)의 평균, 표준편차 등
기초통계량(describe())을 구하시오.

In [ ]:
iris.groupby('species').describe()

sepal_length                                              \
                  count   mean       std  min    25%  50%  75%  max   
species                                                               
setosa             50.0  5.006  0.352490  4.3  4.800  5.0  5.2  5.8   
versicolor         50.0  5.936  0.516171  4.9  5.600  5.9  6.3  7.0   
virginica          50.0  6.588  0.635880  4.9  6.225  6.5  6.9  7.9   

           sepal_width         ... petal_length      petal_width         \
                 count   mean  ...          75%  max       count   mean   
species                        ...                                        
setosa            50.0  3.428  ...        1.575  1.9        50.0  0.246   
versicolor        50.0  2.770  ...        4.600  5.1        50.0  1.326   
virginica         50.0  2.974  ...        5.875  6.9        50.0  2.026   

                                               
                 std  min  25%  50%  75%  max  
species                                        
setosa      0.105386  0.1  0.2  0.2  0.3  0.6  
versicolor  0.197753  1.0  1.2  1.3  1.5  1.8  
virginica   0.274650  1.4  1.8  2.0  2.3  2.5  

[3 rows x 32 columns]

b. 3분위수(Q3)와 1분위수(Q1)의 차이보다 1.5배가 크거나 작은 데이터는 이상치이다.
즉,
Q1 - 1.5 * (Q3 - Q1) 보다 작은 데이터
Q3 + 1.5 * (Q3 - Q1) 보다 큰 데이터
이 이상치를 제거하고 위의 4가지 항목에 대해서 평균, 표준편차를 구하시오.

In [ ]:
#평균
def cal_mean(s):
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    low = q1 - 1.5 * (q3 - q1)
    high = q3 + 1.5 * (q3 - q1)

    c = (s>=low) & (s<=high)
    return s[c].mean()

for col in iris[['sepal_length','sepal_width','petal_width','petal_length']]:
    print(col, cal_mean(iris[col]).round(2))

sepal_length 5.84
sepal_width 3.04
petal_width 1.2
petal_length 3.76


In [ ]:
#표준편차
def cal_std(s):
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    low = q1 - 1.5 * (q3 - q1)
    high = q3 + 1.5 * (q3 - q1)

    c = (s>=low) & (s<=high)
    return s[c].std()

for col in iris[['sepal_length','sepal_width','petal_width','petal_length']]:
    print(col, cal_std(iris[col]).round(2))

sepal_length 0.83
sepal_width 0.4
petal_width 0.76
petal_length 1.77


# 2. Titanic

a. 타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 열인 category1 열을 만드시오.
category1 카테고리는 다음과 같이 정의됨
1) 20살이 넘으면 성별을 그대로 사용한다.
2) 20살 미만이면 성별에 관계없이 “child”라고 한다.

In [ ]:
category1 = titanic[['age','sex']]
category1.head()

,age,sex
0,22.0,male
1,38.0,female
2,26.0,female
3,35.0,female
4,35.0,male


In [ ]:
category1['sex1'] = category1.apply(lambda x: 'child' if x.age < 20 else x.sex ,axis=1)
category1.tail()

<ipython-input-7-197fa26faf7b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category1['sex1'] = category1.apply(lambda x: 'child' if x.age < 20 else x.sex ,axis=1)


,age,sex,sex1
886,27.0,male,male
887,19.0,female,child
888,NaN,female,female
889,26.0,male,male
890,32.0,male,male


b. 타이타닉호의 승객 중 나이를 명시하지 않은 고객은 나이를 명시한 고객의 평균 나이 값
이 되도록 titanic 데이터프레임을 고치시오.

In [ ]:
# 평균 나이값 구하기
most_age = titanic['age'].value_counts(dropna=True).idxmax()
most_age

24.0

In [ ]:
# 채워졌는지 확인
titanic['age'].fillna(most_age, inplace=True)
titanic.isnull().sum()

survived         0
pclass           0
sex              0
age              0
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [ ]:
titanic.head(20)
# index 5, 17, 19번 변경됨 확인 완료.

c. 성별, 선실(class)별, 출발지(embark_town)별 생존율을 구하시오.

In [ ]:
df = titanic.loc[:, ['sex', 'class', 'embark_town', 'survived']]
df

,sex,class,embark_town,survived
0,male,Third,Southampton,0
1,female,First,Cherbourg,1
2,female,Third,Southampton,1
3,female,First,Southampton,1
4,male,Third,Southampton,0
...,...,...,...,...
886,male,Second,Southampton,0
887,female,First,Southampton,1
888,female,Third,Southampton,0
889,male,First,Cherbourg,1


In [ ]:
# 성별, 선실별 생존율
grouped = df.groupby(['class', 'sex'])
average = grouped.mean()
print(average)

               survived
class  sex             
First  female  0.968085
       male    0.368852
Second female  0.921053
       male    0.157407
Third  female  0.500000
       male    0.135447


In [ ]:
# 출발지별 생존율
grouped = df.groupby(['embark_town'])
average = grouped.mean()
print(average)

             survived
embark_town          
Cherbourg    0.553571
Queenstown   0.389610
Southampton  0.336957


d. 타이타닉호 승객을 ‘미성년자’, ‘청년’, ‘중년’, ‘장년’, ‘노년’ 나이 그룹으로 나누고,
각 그룹별 생존율을 구하시오.
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]

In [ ]:
# 그룹 만들기
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]

titanic["agegroup"]=pd.cut(titanic['age'],bins,right=False,labels=labels)
titanic.head()

In [ ]:
# 나이 그룹별 생존율
df2 = titanic.loc[:, ['survived','agegroup']]
grouped2 = df2.groupby(['agegroup'])
average = grouped2.mean()
print(average)

e. qcut 명령으로 세 개의 나이 그룹을 만들고, 나이 그룹별 남녀 성비와 생존율을 구하시오.

# 3. Mile Per Gallon

a. 배기량(displacement) 대비 마력(horsepower) 열(hp_per_cc)을 추가하시오.

In [ ]:
df['hp_per_cc']=0  
df.head()

b. name으로부터 manufacturer(제조사)와 모델을 추출하여
새로운 열 manufacturer와 model을 추가하고, name 열은 삭제하시오.

In [ ]:
df = df.dropna()
df['hp_per_cc'] = df['horsepower']/df['displacement']
df['manufacture'] = 0
df['manufacture'] = df['name'].str.split(' ').str[0]
df['model'] = df['name'].str.split(' ').str[1]

c. 엔진의 실린더(cylinders) 갯수별 연비(mpg)의 평균을 구하시오.

In [ ]:
df['cylinders'].unique()
df_cy = df.groupby('cylinders')
df_cy.mean()

d. 생산지(origin)별 배기량 대비 마력(hp_per_cc)의 평균을 구하시오.

In [ ]:
df['origin'].unique()
df_ori = df.groupby('origin')
df_ori.mean()

e. 모델이 5개 이상인 제조사에 대하여
연비(mpg)의 평균이 가장 좋은 제조사 Top 5를 구하시오.